In [1]:
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel("DataSets/Churn_Modelling.xlsx",index_col=0)
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
df.drop(["CustomerId","Surname","Geography"],axis=1,inplace=True)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,
1,619,Female,42,2,0.00,1,1,1,101348.88,1
2,608,Female,41,1,83807.86,1,0,1,112542.58,0
3,502,Female,42,8,159660.80,3,1,0,113931.57,1
4,699,Female,39,1,0.00,2,0,0,93826.63,0
5,850,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...
9996,771,Male,39,5,0.00,2,1,0,96270.64,0
9997,516,Male,35,10,57369.61,1,1,1,101699.77,0
9998,709,Female,36,7,0.00,1,0,1,42085.58,1


In [6]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [7]:
X = df.iloc[:,:-1]
X.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
RowNumber,,,,,,,,,
1,619,Female,42,2,0.00,1,1,1,101348.88
2,608,Female,41,1,83807.86,1,0,1,112542.58
3,502,Female,42,8,159660.80,3,1,0,113931.57
4,699,Female,39,1,0.00,2,0,0,93826.63
5,850,Female,43,2,125510.82,1,1,1,79084.10


In [8]:
Y = df.Exited
Y

RowNumber
1        1
2        0
3        1
4        0
5        0
        ..
9996     0
9997     0
9998     1
9999     1
10000    0
Name: Exited, Length: 10000, dtype: int64

In [9]:
df.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
CT = ColumnTransformer(transformers = [ ("trans1",OrdinalEncoder(),["Gender"]),
                                        ("trans2",StandardScaler(),['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
                                                                    'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])],
                       remainder='passthrough')
X = CT.fit_transform(X)

In [11]:
X[0]

array([ 0.        , -0.32622142,  0.29351742, -1.04175968, -1.22584767,
       -0.91158349,  0.64609167,  0.97024255,  0.02188649])

In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=1)

In [13]:
def CalModel(model):
    model.fit(X_train,Y_train)
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test,Y_pred))

# Logistic Regression

In [14]:
LOG_REG = LogisticRegression()
CalModel(LOG_REG)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89      2373
           1       0.64      0.17      0.27       627

    accuracy                           0.81      3000
   macro avg       0.73      0.57      0.58      3000
weighted avg       0.78      0.81      0.76      3000



In [15]:
params = [ ['lbfgs','l2'],['lbfgs','none'],
           ['liblinear','l1'],['liblinear','l2'],
           ['newton-cg','l2'],['newton-cg','none'],
           ['sag','l2'],['sag','none'],
           ['saga','l1'],['saga','l2'],['saga','none']  ]

#  ['saga','elasticnet']  --->  Need to perform Scalling on the Data 

all_combinations = []

for i in params:

    from sklearn.linear_model import LogisticRegression

    model = LogisticRegression(solver=i[0] , penalty=i[1])

    model.fit(X_train,Y_train)

    Y_pred = model.predict(X_test)
    
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(Y_test,Y_pred)
    
    print(f"{i} ---> {acc} ")
    
    all_combinations.append(acc)

['lbfgs', 'l2'] ---> 0.8066666666666666 
['lbfgs', 'none'] ---> 0.8066666666666666 
['liblinear', 'l1'] ---> 0.8066666666666666 
['liblinear', 'l2'] ---> 0.8066666666666666 
['newton-cg', 'l2'] ---> 0.8066666666666666 
['newton-cg', 'none'] ---> 0.8066666666666666 
['sag', 'l2'] ---> 0.8066666666666666 
['sag', 'none'] ---> 0.8066666666666666 
['saga', 'l1'] ---> 0.8066666666666666 
['saga', 'l2'] ---> 0.8066666666666666 
['saga', 'none'] ---> 0.8066666666666666 


In [16]:
# Best Hyperparmeter for Logistic Regression
print(f"{params[all_combinations.index(max(all_combinations))]} ---> {max(all_combinations)}")

['lbfgs', 'l2'] ---> 0.8066666666666666


# KNN

In [17]:
KNN = KNeighborsClassifier()
CalModel(KNN)

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      2373
           1       0.69      0.39      0.50       627

    accuracy                           0.84      3000
   macro avg       0.77      0.67      0.70      3000
weighted avg       0.82      0.84      0.82      3000



In [18]:
KNN_LIST = []
for i in range(1,50):
    KNN = KNeighborsClassifier(n_neighbors=i)
    KNN.fit(X_train,Y_train)
    Y_pred = KNN.predict(X_test)
    print(f"{i} ----> {accuracy_score(Y_test,Y_pred)}")
    KNN_LIST.append(accuracy_score(Y_test,Y_pred))

1 ----> 0.7886666666666666
2 ----> 0.8303333333333334
3 ----> 0.8243333333333334
4 ----> 0.835
5 ----> 0.8356666666666667
6 ----> 0.8353333333333334
7 ----> 0.8393333333333334
8 ----> 0.8373333333333334
9 ----> 0.8406666666666667
10 ----> 0.8376666666666667
11 ----> 0.8373333333333334
12 ----> 0.838
13 ----> 0.8406666666666667
14 ----> 0.838
15 ----> 0.8403333333333334
16 ----> 0.8406666666666667
17 ----> 0.8446666666666667
18 ----> 0.8423333333333334
19 ----> 0.8423333333333334
20 ----> 0.8426666666666667
21 ----> 0.8443333333333334
22 ----> 0.84
23 ----> 0.842
24 ----> 0.839
25 ----> 0.8413333333333334
26 ----> 0.8373333333333334
27 ----> 0.8396666666666667
28 ----> 0.835
29 ----> 0.839
30 ----> 0.8356666666666667
31 ----> 0.839
32 ----> 0.8376666666666667
33 ----> 0.8373333333333334
34 ----> 0.8366666666666667
35 ----> 0.8366666666666667
36 ----> 0.8346666666666667
37 ----> 0.836
38 ----> 0.8343333333333334
39 ----> 0.8333333333333334
40 ----> 0.833
41 ----> 0.833
42 ----> 0.833
43 

In [19]:
# Best Hyperparmeter for KNN
print(f"{KNN_LIST.index(max(KNN_LIST))+1} ---> {max(KNN_LIST)}")

17 ---> 0.8446666666666667


# Decision Tree

In [20]:
DTC_GINI = DecisionTreeClassifier()
CalModel(DTC_GINI)

              precision    recall  f1-score   support

           0       0.87      0.86      0.86      2373
           1       0.49      0.50      0.49       627

    accuracy                           0.78      3000
   macro avg       0.68      0.68      0.68      3000
weighted avg       0.79      0.78      0.79      3000



In [21]:
DTC_ENTROPY = DecisionTreeClassifier(criterion="entropy")
CalModel(DTC_ENTROPY)

              precision    recall  f1-score   support

           0       0.87      0.86      0.86      2373
           1       0.49      0.49      0.49       627

    accuracy                           0.79      3000
   macro avg       0.68      0.68      0.68      3000
weighted avg       0.79      0.79      0.79      3000



### MAX_DEPT

In [22]:
MAX_DEPT_LIST = []
for i in range(1,50):
    DTC_1 = DecisionTreeClassifier(max_depth=i,)
    DTC_1.fit(X_train,Y_train)
    Y_pred = DTC_1.predict(X_test)
    print(f"{i} ---> {accuracy_score(Y_test,Y_pred)}")
    MAX_DEPT_LIST.append(accuracy_score(Y_test,Y_pred))

1 ---> 0.791
2 ---> 0.8256666666666667
3 ---> 0.8393333333333334
4 ---> 0.8483333333333334
5 ---> 0.8546666666666667
6 ---> 0.8436666666666667
7 ---> 0.8473333333333334
8 ---> 0.8453333333333334
9 ---> 0.8333333333333334
10 ---> 0.8286666666666667
11 ---> 0.823
12 ---> 0.8213333333333334
13 ---> 0.8076666666666666
14 ---> 0.801
15 ---> 0.7993333333333333
16 ---> 0.7993333333333333
17 ---> 0.793
18 ---> 0.789
19 ---> 0.7853333333333333
20 ---> 0.7856666666666666
21 ---> 0.7846666666666666
22 ---> 0.7876666666666666
23 ---> 0.788
24 ---> 0.786
25 ---> 0.7883333333333333
26 ---> 0.783
27 ---> 0.7866666666666666
28 ---> 0.7853333333333333
29 ---> 0.7873333333333333
30 ---> 0.7826666666666666
31 ---> 0.7886666666666666
32 ---> 0.7866666666666666
33 ---> 0.784
34 ---> 0.7853333333333333
35 ---> 0.7873333333333333
36 ---> 0.7843333333333333
37 ---> 0.7776666666666666
38 ---> 0.784
39 ---> 0.7856666666666666
40 ---> 0.7863333333333333
41 ---> 0.787
42 ---> 0.78
43 ---> 0.784
44 ---> 0.78533333

In [23]:
print(f"{MAX_DEPT_LIST.index(max(MAX_DEPT_LIST))+1} ---> {max(MAX_DEPT_LIST)}")

5 ---> 0.8546666666666667


In [24]:
DTC_MAX_DEPT_LIST = DecisionTreeClassifier(max_depth=5)
CalModel(DTC_MAX_DEPT_LIST)

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      2373
           1       0.79      0.41      0.54       627

    accuracy                           0.85      3000
   macro avg       0.83      0.69      0.73      3000
weighted avg       0.85      0.85      0.84      3000



### MIN_SAMPLE_SPLIT

In [25]:
MIN_SAMPLE_SPLIT = []
for i in range(2,50):
    DTC_2 = DecisionTreeClassifier(min_samples_split=i)
    DTC_2.fit(X_train,Y_train)
    Y_pred = DTC_2.predict(X_test)
    
    print(f"{i} ---> {accuracy_score(Y_test,Y_pred)}")
    
    MIN_SAMPLE_SPLIT.append(accuracy_score(Y_test,Y_pred))

2 ---> 0.7816666666666666
3 ---> 0.7896666666666666
4 ---> 0.786
5 ---> 0.7903333333333333
6 ---> 0.7893333333333333
7 ---> 0.7893333333333333
8 ---> 0.7886666666666666
9 ---> 0.7926666666666666
10 ---> 0.7933333333333333
11 ---> 0.7946666666666666
12 ---> 0.7973333333333333
13 ---> 0.7973333333333333
14 ---> 0.799
15 ---> 0.8016666666666666
16 ---> 0.8016666666666666
17 ---> 0.8076666666666666
18 ---> 0.808
19 ---> 0.81
20 ---> 0.813
21 ---> 0.814
22 ---> 0.814
23 ---> 0.8156666666666667
24 ---> 0.8173333333333334
25 ---> 0.8233333333333334
26 ---> 0.8216666666666667
27 ---> 0.823
28 ---> 0.8233333333333334
29 ---> 0.821
30 ---> 0.82
31 ---> 0.8196666666666667
32 ---> 0.8206666666666667
33 ---> 0.821
34 ---> 0.8206666666666667
35 ---> 0.822
36 ---> 0.8226666666666667
37 ---> 0.8166666666666667
38 ---> 0.821
39 ---> 0.8176666666666667
40 ---> 0.818
41 ---> 0.8196666666666667
42 ---> 0.817
43 ---> 0.82
44 ---> 0.8176666666666667
45 ---> 0.819
46 ---> 0.8223333333333334
47 ---> 0.822
48 

In [26]:
print(f"{MIN_SAMPLE_SPLIT.index(max(MIN_SAMPLE_SPLIT))+2} ---> {max(MIN_SAMPLE_SPLIT)}")

25 ---> 0.8233333333333334


In [27]:
DTC_MIN_SAMPLE_SPLIT = DecisionTreeClassifier(min_samples_split=25)
CalModel(DTC_MIN_SAMPLE_SPLIT)

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      2373
           1       0.59      0.47      0.52       627

    accuracy                           0.82      3000
   macro avg       0.73      0.69      0.71      3000
weighted avg       0.81      0.82      0.81      3000



### MIN_SAMPLE_LEAF

In [28]:
MIN_SAMPLE_LEAF = []
for i in range(1,50):
    DTC_3 = DecisionTreeClassifier(min_samples_leaf=i)
    DTC_3.fit(X_train,Y_train)
    Y_pred = DTC_3.predict(X_test)
    
    print(f"{i} ---> {accuracy_score(Y_test,Y_pred)}")
    
    MIN_SAMPLE_LEAF.append(accuracy_score(Y_test,Y_pred))

1 ---> 0.7876666666666666
2 ---> 0.8003333333333333
3 ---> 0.7983333333333333
4 ---> 0.8133333333333334
5 ---> 0.818
6 ---> 0.8186666666666667
7 ---> 0.8263333333333334
8 ---> 0.823
9 ---> 0.8286666666666667
10 ---> 0.8266666666666667
11 ---> 0.8323333333333334
12 ---> 0.836
13 ---> 0.8373333333333334
14 ---> 0.84
15 ---> 0.8343333333333334
16 ---> 0.8353333333333334
17 ---> 0.838
18 ---> 0.8376666666666667
19 ---> 0.835
20 ---> 0.84
21 ---> 0.8403333333333334
22 ---> 0.8433333333333334
23 ---> 0.8463333333333334
24 ---> 0.8473333333333334
25 ---> 0.8483333333333334
26 ---> 0.8446666666666667
27 ---> 0.8453333333333334
28 ---> 0.8443333333333334
29 ---> 0.8443333333333334
30 ---> 0.8443333333333334
31 ---> 0.847
32 ---> 0.8443333333333334
33 ---> 0.8443333333333334
34 ---> 0.842
35 ---> 0.835
36 ---> 0.837
37 ---> 0.8356666666666667
38 ---> 0.8356666666666667
39 ---> 0.8356666666666667
40 ---> 0.8383333333333334
41 ---> 0.8366666666666667
42 ---> 0.84
43 ---> 0.84
44 ---> 0.84266666666

In [29]:
print(f"{MIN_SAMPLE_LEAF.index(max(MIN_SAMPLE_LEAF))+2} ---> {max(MIN_SAMPLE_LEAF)}")

26 ---> 0.8483333333333334


In [30]:
DTC_MIN_SAMPLE_LEAF = DecisionTreeClassifier(min_samples_split=25)
CalModel(DTC_MIN_SAMPLE_LEAF)

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      2373
           1       0.59      0.46      0.52       627

    accuracy                           0.82      3000
   macro avg       0.73      0.69      0.71      3000
weighted avg       0.81      0.82      0.81      3000



# Stratified K-FOLD on Decision Tree

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

SKFOLD_LIST = []

for i in range(2,20):
    skfolds = StratifiedKFold(n_splits=i)

    DTC_SKFOLD = DecisionTreeClassifier()

    scores = cross_val_score(DTC_SKFOLD,X,Y,cv=skfolds)
    
    SKFOLD_LIST.append(np.mean(scores))



In [32]:
print(f"{SKFOLD_LIST.index(max(SKFOLD_LIST))+2} ---> {max(SKFOLD_LIST)}")

10 ---> 0.7866000000000001
